<a href="https://colab.research.google.com/github/igornunespatricio/BairesDev-Machine-Learning-Practitioner/blob/main/virtual_assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import subprocess

# List of libraries to install
libraries = [
    "gtts",
    "transformers",
    "speechrecognition",
    "pydub"
]

# Function to install libraries while suppressing output
def install_libraries():
    for library in libraries:
        subprocess.run(f"pip install {library} > /dev/null 2>&1", shell=True)

# Function to print the version of each installed library
def print_library_versions():
    for library in libraries:
        try:
            # Print the version using 'pip show'
            result = subprocess.run(f"pip show {library}", shell=True, capture_output=True, text=True)
            # Extract and print the version
            for line in result.stdout.splitlines():
                if line.startswith("Version"):
                    print(f"{library}: {line}")
        except Exception as e:
            print(f"Error retrieving version for {library}: {e}")

# Install the libraries
install_libraries()

# Print the versions
print_library_versions()

gtts: Version: 2.5.4
transformers: Version: 4.46.3
speechrecognition: Version: 3.12.0
pydub: Version: 0.25.1


In [2]:
from gtts import gTTS
from transformers import pipeline
import speech_recognition as sr
from pydub import AudioSegment
import os
from IPython.display import Audio, Javascript
import ast

from google.colab import output
from base64 import b64decode
from io import BytesIO

import subprocess
import time

In [3]:
# Create necessary directories if they don't exist
base_dir = "processing_files"
audio_dir = os.path.join(base_dir, "audio_files")
text_dir = os.path.join(base_dir, "text_files")
processed_text_dir = os.path.join(base_dir, "processed_text_files")
emotion_dir = os.path.join(base_dir, "emotion_files")


for directory in [audio_dir, text_dir, processed_text_dir, emotion_dir]:
    if not os.path.exists(directory):
        os.makedirs(directory)

In [4]:
# Initialize the recognizer for speech recognition
r = sr.Recognizer()

# Load the pre-trained emotion detection model
emotion_analyzer = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [5]:
# Function to save raw text into a file
def save_text_to_file(text, idx):
    """
    Save raw text into a file in the 'text_files' directory.
    """
    text_file_path = os.path.join(text_dir, f"text_{idx}.txt")
    with open(text_file_path, 'w') as f:
        f.write(text)
    print(f"Saved raw text for text_{idx} at {text_file_path}")
    return text_file_path

# Function to convert text to audio and save it as an MP3
def process_text_to_audio(idx):
    """
    Read text from a file, convert it to audio, and save as an MP3.
    """
    # Read the raw text from the file
    text_file_path = os.path.join(text_dir, f"text_{idx}.txt")
    try:
        with open(text_file_path, 'r') as f:
            text = f.read()
    except FileNotFoundError:
        print(f"Text file text_{idx}.txt not found.")
        return None

    # Convert text to speech (audio)
    tts = gTTS(text=text, lang='en')
    audio_path = os.path.join(audio_dir, f"audio_{idx}.mp3")
    tts.save(audio_path)
    print(f"Saved audio file for text_{idx} at {audio_path}")
    return audio_path

# Function to convert audio (MP3) to text (transcription)
def process_audio_to_text(audio_path, idx):
    """
    Convert an audio file to text using speech recognition.
    """
    # Convert MP3 to WAV
    wav_path = audio_path.replace(".mp3", ".wav")
    try:
        sound = AudioSegment.from_mp3(audio_path)
        sound.export(wav_path, format="wav")
        print(f"Converted {audio_path} to {wav_path}")
    except FileNotFoundError:
        print(f"Error: {audio_path} not found.")
        return None

    # Process the audio file and extract text using Speech Recognition
    with sr.AudioFile(wav_path) as source:
        audio = r.record(source)

    try:
        # Convert audio to text
        text_from_audio = r.recognize_google(audio)
        print(f"Transcription for audio_{idx}: {text_from_audio}")

        # Save the processed text to a file
        processed_text_file_path = os.path.join(processed_text_dir, f"processed_text_{idx}.txt")
        with open(processed_text_file_path, 'w') as f:
            f.write(text_from_audio)
        print(f"Saved processed text for audio_{idx} at {processed_text_file_path}")

        return text_from_audio
    except sr.UnknownValueError:
        print(f"Speech Recognition could not understand audio_{idx}")
        return None
    except sr.RequestError as e:
        print(f"Could not request results from Speech Recognition service for audio_{idx}: {e}")
        return None

# Function to classify the emotion of a given text and save the result
def process_text_to_emotion(text, idx):
    """
    Classify the emotion of the given text using a pre-trained model and save the result.
    """
    # Predict the emotion from the transcribed text
    result = emotion_analyzer(text)
    print(f"Emotion Prediction for text_{idx}: {result}")

    # Save the emotion classification to a file
    emotion_file_path = os.path.join(emotion_dir, f"emotion_{idx}.txt")
    with open(emotion_file_path, 'w') as f:
        f.write(str(result))
    print(f"Saved emotion classification for text_{idx} at {emotion_file_path}")

    return result

# Function to process multiple texts from raw text files
def process_texts():
    """
    Process all raw text files, generate audio, transcribe, and classify emotion.
    """
    text_files = [f for f in os.listdir(text_dir) if f.endswith('.txt')]

    for idx, text_file in enumerate(text_files, start=1):
        # 1. Convert text to audio and save it
        audio_path = process_text_to_audio(idx)

        # 2. Convert audio to text and save it
        if audio_path:
            transcribed_text = process_audio_to_text(audio_path, idx)

            # 3. If transcription is successful, classify emotion and save it
            if transcribed_text:
                process_text_to_emotion(transcribed_text, idx)

In [6]:
# Example: Create sample raw text files
texts = [
    "I am so excited about the new technology!",
    "I am really angry right now, this is frustrating!",
    "It's such a sad day. I feel so low.",
    "I am so happy to see you! This is amazing!",
    "I am scared about the future and what it holds.",
    "Surprise! I didn't expect this to happen!",
    "I'm feeling a bit stressed with all the work piling up.",
    "This is just unbelievable! I'm shocked!",
    "I'm really nervous about this presentation.",
    "I feel peaceful and calm today.",
]

# Save these texts as raw files
for idx, text in enumerate(texts, start=1):
    save_text_to_file(text, idx)

Saved raw text for text_1 at processing_files/text_files/text_1.txt
Saved raw text for text_2 at processing_files/text_files/text_2.txt
Saved raw text for text_3 at processing_files/text_files/text_3.txt
Saved raw text for text_4 at processing_files/text_files/text_4.txt
Saved raw text for text_5 at processing_files/text_files/text_5.txt
Saved raw text for text_6 at processing_files/text_files/text_6.txt
Saved raw text for text_7 at processing_files/text_files/text_7.txt
Saved raw text for text_8 at processing_files/text_files/text_8.txt
Saved raw text for text_9 at processing_files/text_files/text_9.txt
Saved raw text for text_10 at processing_files/text_files/text_10.txt


In [7]:
# Process all the text files
process_texts()

Saved audio file for text_1 at processing_files/audio_files/audio_1.mp3
Converted processing_files/audio_files/audio_1.mp3 to processing_files/audio_files/audio_1.wav
Transcription for audio_1: I am so excited about the new technology
Saved processed text for audio_1 at processing_files/processed_text_files/processed_text_1.txt
Emotion Prediction for text_1: [{'label': 'joy', 'score': 0.9765963554382324}]
Saved emotion classification for text_1 at processing_files/emotion_files/emotion_1.txt
Saved audio file for text_2 at processing_files/audio_files/audio_2.mp3
Converted processing_files/audio_files/audio_2.mp3 to processing_files/audio_files/audio_2.wav
Transcription for audio_2: I am really angry right now this is frustrating
Saved processed text for audio_2 at processing_files/processed_text_files/processed_text_2.txt
Emotion Prediction for text_2: [{'label': 'anger', 'score': 0.986971914768219}]
Saved emotion classification for text_2 at processing_files/emotion_files/emotion_2.tx

In [35]:
# Utility function to read file content (to avoid repetition)
def read_file_content(file_path):
    """Reads the content of a file and returns it."""
    try:
        with open(file_path, 'r') as f:
            return f.read()
    except FileNotFoundError:
        print(f"File {file_path} not found.")
        return None

# Function to get raw text from the file
def get_raw_text(idx):
    text_file_path = os.path.join(text_dir, f"text_{idx}.txt")
    return read_file_content(text_file_path)

# Function to get processed text (transcribed from audio)
def get_processed_text(idx):
    processed_text_file_path = os.path.join(processed_text_dir, f"processed_text_{idx}.txt")
    return read_file_content(processed_text_file_path)

# Function to get emotion from processed text
def get_emotion(idx):
    emotion_file_path = os.path.join(emotion_dir, f"emotion_{idx}.txt")
    return read_file_content(emotion_file_path)

def speak(text, path, wait=5):
    tts = gTTS(text=text, lang='en')
    tts.save(path)
    display(Audio(path, autoplay=True))
    time.sleep(wait)

# Function to display all the requested information for a given idx
def display_details_for_idx(idx):
    # Get and display raw text
    raw_text = get_raw_text(idx)
    if not raw_text:
        return

    print(f"Raw Text for idx={idx}: {raw_text}")

    # Get and display processed text
    processed_text = get_processed_text(idx)
    if not processed_text:
        return

    print(f"Processed Text for idx={idx}: {processed_text}")

    # Get and display emotion classification
    emotion = get_emotion(idx)

    if not emotion:
        return

    emotion_dict = ast.literal_eval(emotion)[0]
    print(f"Emotion for idx={idx}: {emotion}")
    text_with_emotion = f"""
    The index selected was {idx}.
    The text is: {processed_text}.
    The emotion is {emotion_dict['label']} with score {round(emotion_dict['score'], 2)}
    """
    speak(text_with_emotion, path='temp_audio.mp3')

# Example: Request details for a specific idx (e.g., idx=1)
def request_detailed_idx():
    idx = int(input("Enter the idx to retrieve details (e.g., 1, 2, 3...): "))
    display_details_for_idx(idx)

In [32]:
request_detailed_idx()

Enter the idx to retrieve details (e.g., 1, 2, 3...): 7
Raw Text for idx=7: I'm feeling a bit stressed with all the work piling up.
Processed Text for idx=7: I'm feeling a bit stressed with all the work piling up
Emotion for idx=7: [{'label': 'anger', 'score': 0.9646212458610535}]


In [12]:
RECORD = """
const sleep = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise((resolve, reject) => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.target.result)
  reader.onerror = e => reject(new Error("Failed to read blob"))
  reader.readAsDataURL(blob)
})
var recordUntilSilence = time => new Promise(async (resolve, reject) => {
  let stream, recorder, chunks, blob, text, audioContext, analyser, dataArr, silenceStart, threshold = 50, silenceDelay = 2000
  try {
    stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  } catch (err) {
    return reject(new Error("Failed to get media stream"))
  }
  audioContext = new AudioContext()
  const source = audioContext.createMediaStreamSource(stream)
  analyser = audioContext.createAnalyser()
  analyser.fftSize = 512
  dataArr = new Uint8Array(analyser.frequencyBinCount)
  source.connect(analyser)
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.onstop = async () => {
    blob = new Blob(chunks)
    try {
      text = await b2text(blob)
      resolve(text)
    } catch (err) {
      reject(new Error("Failed to convert blob to text"))
    }
  }
  recorder.onerror = e => reject(new Error("Recorder error"))
  recorder.start()
  const checkSilence = () => {
    analyser.getByteFrequencyData(dataArr)
    const avg = dataArr.reduce((p, c) => p + c, 0) / dataArr.length

    if (avg < threshold) {
      if (silenceStart === null) silenceStart = new Date().getTime()
      else if (new Date().getTime() - silenceStart > silenceDelay) {
        recorder.stop()
        audioContext.close()
        return
      }
    } else {
      silenceStart = null
    }
    requestAnimationFrame(checkSilence)
  }
  silenceStart = null
  checkSilence()
})
console.log("JavaScript code executed successfully.")
"""

def record_until_silence():
  try:
    display(Javascript(RECORD))
    s = output.eval_js('recordUntilSilence()')
    b = b64decode(s.split(',')[1])
    audio = AudioSegment.from_file(BytesIO(b))
    return audio
  except Exception as e:
    print(f"An error occurred: {e}")
    return None

In [38]:
order_path = 'order_audio.mp3'
failed_audio_path = 'failed_audio.mp3'


def virtual_assistant():
    speak("Run process text or request detailed index", path='order_audio.mp3', wait=3)
    audio = record_until_silence()

    if audio:
        audio.export("recorded_audio.wav", format="wav")
        with sr.AudioFile("recorded_audio.wav") as source:
          audio_data = r.record(source)

        try:
            text = r.recognize_google(audio_data)
            print("You said:", text)
            if "process text" in text.lower():
                process_texts()
            elif "request detailed index" in text.lower():
                request_detailed_idx()
            else:
                text = "I did not understand your request. Please try again."
                speak(text, path=failed_audio_path, wait=10)
                virtual_assistant()
        except sr.UnknownValueError:
            print("Speech recognition could not understand audio")
            speak("I could not understand your audio. Please try again.", path=failed_audio_path)
            virtual_assistant()
        except sr.RequestError as e:
            print(f"Could not request results from speech recognition service; {e}")
            speak("There was an error with the speech recognition service.", path=failed_audio_path)

virtual_assistant()

<IPython.core.display.Javascript object>

You said: Apple movie


<IPython.core.display.Javascript object>

You said: request detailed index
Enter the idx to retrieve details (e.g., 1, 2, 3...): 3
Raw Text for idx=3: It's such a sad day. I feel so low.
Processed Text for idx=3: it's such a sad day I feel so low
Emotion for idx=3: [{'label': 'sadness', 'score': 0.991736114025116}]
